<a href="https://colab.research.google.com/github/Yash-1812/Advance_Python/blob/main/BAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


from google.colab import files
uploaded = files.upload()

import zipfile, os

# Use uploaded filename automatically
zip_path = list(uploaded.keys())[0]
print("Using ZIP:", zip_path)

output_dir = "/content/extracted_slides"
os.makedirs(output_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as z:
    z.extractall(output_dir)

print("Extracted to:", output_dir)

# Show extracted files
for root, dirs, files_list in os.walk(output_dir):
    for f in files_list:
        print(os.path.join(root, f))

Saving Slides-20251114T131049Z-1-001.zip to Slides-20251114T131049Z-1-001.zip
Using ZIP: Slides-20251114T131049Z-1-001.zip
Extracted to: /content/extracted_slides
/content/extracted_slides/Slides/10. Mergers and Acquisitions.pptx
/content/extracted_slides/Slides/7. Credit risk analysis.pdf
/content/extracted_slides/Slides/8. Relative Valuation.pptx
/content/extracted_slides/Slides/11. Mergers and Acquisitions.pptx
/content/extracted_slides/Slides/7. Financial Analysis.pptx
/content/extracted_slides/Slides/3. Accounting Analysis.pptx
/content/extracted_slides/Slides/1. PPT 1.pptx


Extracting Text from pdf and pptx files into txt format

In [2]:


!pip install --quiet python-pptx PyPDF2

import os, pathlib, json
from pptx import Presentation
import PyPDF2


BASE_DIR = "/content/extracted_slides/Slides"
OUT_DIR = "/content/extracted_text"
os.makedirs(OUT_DIR, exist_ok=True)

print("Reading slides from:", BASE_DIR)
print("Saving extracted text to:", OUT_DIR)

manifest = []

def extract_text_from_pptx(path):
    prs = Presentation(path)
    all_text = []

    for slide_idx, slide in enumerate(prs.slides, start=1):
        slide_text = []
        for shape in slide.shapes:
            if hasattr(shape, "text"): # hasattr -> "has attribute" -- here - inspect shape for "text"
                text = shape.text.strip()
                if text:
                    slide_text.append(text)
        if slide_text:
            all_text.append(f"[[SLIDE {slide_idx}]]\n" + "\n".join(slide_text))

    return "\n\n".join(all_text)


def extract_text_from_pdf(path):
    pages = []
    with open(path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for pg, page in enumerate(reader.pages, start=1):
            try:
                text = page.extract_text()
                if text:
                    pages.append(f"[[PAGE {pg}]]\n{text}")
                else:
                    pages.append(f"[[PAGE {pg}]]\n[NO TEXT EXTRACTED]")
            except:
                pages.append(f"[[PAGE {pg}]]\n[ERROR READING PAGE]")
    return "\n\n".join(pages)


for root, dirs, files in os.walk(BASE_DIR):
    for file in files:
        path = os.path.join(root, file)
        ext = pathlib.Path(file).suffix.lower()

        if ext not in [".pptx", ".pdf"]:
            continue

        print("Extracting:", file)

        try:
            if ext == ".pptx":
                text = extract_text_from_pptx(path)
            else:
                text = extract_text_from_pdf(path)

            # save as .txt
            out_path = os.path.join(
                OUT_DIR, pathlib.Path(file).stem + ".txt"
            )
            with open(out_path, "w", encoding="utf-8") as f:
                f.write(text)

            manifest.append({
                "source_file": path,
                "text_file": out_path,
                "original_size": os.path.getsize(path)
            })

        except Exception as e:
            print("Failed to extract:", file, "Error:", e)

manifest_path = os.path.join(OUT_DIR, "text_manifest.json")
with open(manifest_path, "w", encoding="utf-8") as mf:
    json.dump(manifest, mf, indent=2)

print("\n==== EXTRACTION COMPLETE ====")
print("Extracted TXT files are in:", OUT_DIR)
print("Manifest saved to:", manifest_path)

for item in manifest:
    print("✓", item["text_file"])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 14.4 MB/s eta 0:00:00
Reading slides from: /content/extracted_slides/Slides
Saving extracted text to: /content/extracted_text
Extracting: 10. Mergers and Acquisitions.pptx
Extracting: 7. Credit risk analysis.pdf
Extracting: 8. Relative Valuation.pptx
Extracting: 11. Mergers and Acquisitions.pptx
Extracting: 7. Financial Analysis.pptx
Extracting: 3. Accounting Analysis.pptx
Extracting: 1. PPT 1.pptx

==== EXTRACTION COMPLETE ====
Extracted TXT files are in: /content/extracted_text
Manifest saved to: /content/extracted_text/text_manifest.json
✓ /content/extracted_text/10. Mergers and Acquisitions.txt
✓ /content/extracted_text/7. Credit risk analysis.txt
✓ /content/extracted_text/8. Relative Valuation.txt
✓ /content/extracted_text/11. Mergers and Acquisitions.txt
✓ /conte

Chunking Segment

Three functions are created in this segment.

--> One for page/slide based chunking

--> The other for chunking with overlap, to get relation between segments

--> The third for sentence based chuncking

In [4]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:

!pip install --quiet nltk

import os, json, re, nltk
from nltk.tokenize import sent_tokenize
from pathlib import Path

nltk.download('punkt')

TEXT_DIR = "/content/extracted_text"
OUT_DIR = "/content/rag_chunks"
os.makedirs(OUT_DIR, exist_ok=True)

print("Reading text from:", TEXT_DIR)
print("Saving chunks to:", OUT_DIR)


# Helper: clean text

def clean_text(text):
    text = text.replace("\uf0b7", "")   # bullet artifacts
    text = re.sub(r'\n{3,}', '\n\n', text)  # limit excessive newlines
    text = text.strip()
    return text


# Strategy A: Slide / Page based chunking

def chunk_by_markers(text, source_name):
    # Markers are like: [[SLIDE 5]] or [[PAGE 3]]
    pattern = r"\[\[(SLIDE|PAGE)\s+\d+\]\]"
    parts = re.split(pattern, text)

    chunks = []
    slide_num = None
    for i in range(0, len(parts)-1, 2):
        marker_type = parts[i]
        content = parts[i+1].strip()
        if not content:
            continue

        chunks.append({
            "id": f"{source_name}_slidepage_{i}",
            "source": source_name,
            "slide_or_page": marker_type,
            "text": content
        })
    return chunks


# Strategy B: Fixed-size chunks with overlap

def chunk_fixed(text, source_name, chunk_size=800, overlap=150):
    text = clean_text(text)
    chunks = []

    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]

        chunks.append({
            "id": f"{source_name}_fixed_{start}",
            "source": source_name,
            "start": start,
            "end": end,
            "text": chunk
        })

        start = end - overlap

    return chunks

# Sentence based chunking

def chunk_sentence_based(text, source_name, max_sentences=5):
    text = clean_text(text)
    sentences = sent_tokenize(text)

    chunks = []
    for i in range(0, len(sentences), max_sentences):
        chunk_sentences = sentences[i:i+max_sentences]
        chunk_text = " ".join(chunk_sentences).strip()

        chunks.append({
            "id": f"{source_name}_sent_{i}",
            "source": source_name,
            "text": chunk_text
        })

    return chunks

all_chunks = []

for file in os.listdir(TEXT_DIR):
    if not file.endswith(".txt"):
        continue

    source_path = os.path.join(TEXT_DIR, file)
    source_name = Path(file).stem

    print("Chunking:", file)

    with open(source_path, "r", encoding="utf-8") as f:
        raw_text = f.read()

    raw_text = clean_text(raw_text)

    # Using all three stratergies
    chunks_A = chunk_by_markers(raw_text, source_name)
    chunks_B = chunk_fixed(raw_text, source_name, chunk_size=900, overlap=200)
    chunks_C = chunk_sentence_based(raw_text, source_name, max_sentences=4)

    all_chunks.extend(chunks_A)
    all_chunks.extend(chunks_B)
    all_chunks.extend(chunks_C)

# Save chunks to JSONL
chunks_path = os.path.join(OUT_DIR, "chunks.jsonl")
with open(chunks_path, "w", encoding="utf-8") as f:
    for ch in all_chunks:
        f.write(json.dumps(ch) + "\n")

print("\n==== CHUNKING COMPLETE ====")
print("Total chunks:", len(all_chunks))
print("Saved to:", chunks_path)

Reading text from: /content/extracted_text
Saving chunks to: /content/rag_chunks
Chunking: 8. Relative Valuation.txt
Chunking: 7. Financial Analysis.txt
Chunking: 10. Mergers and Acquisitions.txt
Chunking: 11. Mergers and Acquisitions.txt
Chunking: 7. Credit risk analysis.txt
Chunking: 3. Accounting Analysis.txt
Chunking: 1. PPT 1.txt

==== CHUNKING COMPLETE ====
Total chunks: 512
Saved to: /content/rag_chunks/chunks.jsonl


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:

# The RAG chuncks will be present at /content/rag_chunks/chunks.jsonl


!pip install --quiet sentence-transformers faiss-cpu transformers accelerate

!pip install --quiet "transformers[torch]"

# 2) IMPORTS
import os, json, math, time
from pathlib import Path
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch
from tqdm import tqdm

# Paths and settings
CHUNKS_PATH = "/content/rag_chunks/chunks.jsonl"
EMBED_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  # Embedded model
GEN_MODEL_NAME = "google/flan-t5-base"                       # We can also try the t5-large
INDEX_DIR = "/content/faiss_index"
os.makedirs(INDEX_DIR, exist_ok=True)

# parameters can be tweaked
EMBED_BATCH = 64
EMBED_DIM = 384
TOP_K = 5
SEED = 42

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device for generation:", device)

# Loading chunks
chunks = []
with open(CHUNKS_PATH, "r", encoding="utf-8") as f:
    for line in f:
        try:
            chunks.append(json.loads(line))
        except:
            pass

print("Loaded chunks:", len(chunks))

# Using FAISS embeddings
index_path = os.path.join(INDEX_DIR, "faiss.index")
meta_path = os.path.join(INDEX_DIR, "metadata.jsonl")

if os.path.exists(index_path) and os.path.exists(meta_path):
    print("Loading existing FAISS index and metadata...")
    index = faiss.read_index(index_path)
    metadata = [json.loads(line) for line in open(meta_path, "r", encoding="utf-8")]
else:
    print("Creating embeddings and FAISS index from chunks...")

    # loading model
    embedder = SentenceTransformer(EMBED_MODEL_NAME, device=device)
    embedder.max_seq_length = 512


    texts = [ (c.get("text",""), c) for c in chunks ]
    all_texts = [t for t,_ in texts]

    # batch embed
    embeddings = []
    for i in tqdm(range(0, len(all_texts), EMBED_BATCH), desc="Embedding batches"):
        batch_texts = all_texts[i:i+EMBED_BATCH]
        emb = embedder.encode(batch_texts, show_progress_bar=False, convert_to_numpy=True)
        embeddings.append(emb)
    embeddings = np.vstack(embeddings).astype("float32")
    print("Embeddings shape:", embeddings.shape)

    # build FAISS index (IndexFlatIP with normalized vectors for cosine)
    faiss.normalize_L2(embeddings)  # normalize for cosine-sim using inner product
    index = faiss.IndexFlatIP(EMBED_DIM)
    index.add(embeddings)
    print("FAISS index built. n_total =", index.ntotal)

    # save index and metadata
    faiss.write_index(index, index_path)
    with open(meta_path, "w", encoding="utf-8") as mf:
        for c in chunks:
            mf.write(json.dumps(c, ensure_ascii=False) + "\n")
    metadata = chunks.copy()
    print("Saved index to", index_path)
    print("Saved metadata to", meta_path)

# 6) LOAD GENERATOR (seq2seq)
print("Loading generator model:", GEN_MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(GEN_MODEL_NAME).to(device)

# small helper: retrieve top-k and return chunks with scores
def retrieve(query, top_k=TOP_K):
    # embed query with the same embedder used earlier
    global embedder
    if 'embedder' not in globals():
        # lazy load embedder if not present
        embedder = SentenceTransformer(EMBED_MODEL_NAME, device=device)
        embedder.max_seq_length = 512

    q_emb = embedder.encode([query], convert_to_numpy=True)
    q_emb = q_emb.astype("float32")
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)
    results = []
    for score, idx in zip(D[0], I[0]):
        md = metadata[idx]
        results.append({
            "score": float(score),
            "chunk_id": md.get("id"),
            "source": md.get("source"),
            "text": md.get("text", "")[:3000]  # cap preview length
        })
    return results

#  (Optional) Rerank using cross-encoder  (heavy)
# from sentence_transformers import CrossEncoder
# reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device=device)
# def rerank(query, candidates):
#     pairs = [[query, c['text']] for c in candidates]
#     scores = reranker.predict(pairs)
#     for c,s in zip(candidates, scores):
#         c['rerank_score'] = float(s)
#     candidates = sorted(candidates, key=lambda x: x['rerank_score'], reverse=True)
#     return candidates

#  Prompt
def generate_answer(query, top_k=TOP_K, max_length=256):
    # retrieve
    results = retrieve(query, top_k=top_k)
    # optional rerank:
    # results = rerank(query, results)

    # context string from chunks
    context_blocks = []
    for i, r in enumerate(results, start=1):
        block = f"[SOURCE {i}] {r['source']}\n{r['text']}"
        context_blocks.append(block)

    context = "\n\n".join(context_blocks)


    prompt = (
        "You are an assistant answering questions using the provided context. "
        "Use only the facts in the context to answer and cite the sources in brackets. "
        "If the answer isn't in the context, say 'I don't know'.\n\n"
        f"CONTEXT:\n{context}\n\nQUESTION: {query}\n\nAnswer:"
    )

    # tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=4,
            early_stopping=True,
            no_repeat_ngram_size=3
        )
    ans = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return {"answer": ans, "retrieved": results, "prompt": prompt}

# 9) Simple interactive query loop (or use programmatically)
print("\nRAG system ready. Try calling generate_answer(query).\n")
print("Example:")
example_q = "What is relative valuation and when is it useful?"
res = generate_answer(example_q, top_k=4)
print("ANSWER:\n", res['answer'])
print("\nRETRIEVED SOURCES:")
for r in res['retrieved']:
    print(" -", r['source'], "| score:", round(r['score'], 3))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.9 MB/s eta 0:00:00
Device for generation: cuda
Loaded chunks: 512
Creating embeddings and FAISS index from chunks...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]


RAG system ready. Try calling generate_answer(query).

Example:
ANSWER:
 In relative valuation, an asset is valued on the basis of how similar assets are currently priced in the market

RETRIEVED SOURCES:
 - 8. Relative Valuation | score: 0.825
 - 8. Relative Valuation | score: 0.754
 - 1. PPT 1 | score: 0.637
 - 1. PPT 1 | score: 0.631
